In [1]:
!git clone https://github.com/ultralytics/yolov5 ../yolov5

fatal: целевой путь «../yolov5» уже существует и не является пустым каталогом.


In [33]:
import torch
import numpy as np
import os, sys
from PIL import Image
import cv2
import pandas as pd
import math
from torch import nn
from torchvision import models
from torchvision import transforms

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
sys.path.append('../yolov5/')

In [5]:
# yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path='../models/detect/yolo_finetuned_v1.pt')

In [6]:
yolo_model = torch.load('../models/detect/yolo_finetuned_v2.pt')['model'].float().eval().autoshape()

In [7]:
yolo_model = yolo_model.to(device)

In [8]:
# yolo_model(np.random.randn(3, 640, 640)).pandas().xywh[0]

In [9]:
class MultiOutputModel(nn.Module):
    def __init__(self, n_color_classes=3, n_tail_classes=2):
        super().__init__()
        self.resnet = models.resnet34(pretrained=False)
        self.base_model = nn.Sequential(
            *(list(self.resnet.children())[:-1])
        )  # take the model without classifier

        last_channel = (
            models.resnet34().fc.in_features
        )  # size of the layer before the classifier

        # create separate classifiers for our outputs
        self.color = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=last_channel, out_features=n_color_classes),
        )
        self.tail = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=last_channel, out_features=n_tail_classes),
        )

    def forward(self, x):
        x = self.base_model(x)

        # reshape from [batch, channels, 1, 1] to [batch, channels] to put it into classifier
        x = torch.flatten(x, start_dim=1)
        return {
            "color": self.color(x),
            "tail": self.tail(x),
        }

In [10]:
classifier_model = MultiOutputModel()

In [11]:
classifier_model = classifier_model.to(device)

In [12]:
weigths = torch.load('../models/classifier/resnet.pt')['model']

In [13]:
classifier_model.load_state_dict(weigths)

<All keys matched successfully>

In [14]:
tr_pipe = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225)),
])

In [15]:
# tr_pipe(torch.from_numpy(np.random.rand(3, 640, 640).astype(np.uint8)))

In [16]:
images_folder = '../test_images/'

In [17]:
images = [os.path.join(images_folder, path) for path in os.listdir(images_folder)]
images[:5]

['../test_images/А94.jpg',
 '../test_images/С69.jpg',
 '../test_images/С47.jpg',
 '../test_images/B60.jpg',
 '../test_images/А41.jpg']

In [83]:
image.split('/')

['..', 'test_images', 'С10.jpg']

In [88]:
all_dicts = []
for image in images:
    attr_dict = {'filename': image.split('/')[-1], 'is_animal_there': 0, 'is_it_a_dog': 0, 'is_the_owner_there': 0, 'color': 0, 'tail': 0}
    cv_image = cv2.imread(image)
    cv_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
    res_df = yolo_model(cv_image).pandas().xywh[0]
    animals = res_df.query("name=='dog' or name=='cat' or name=='bird'")
    if len(animals) > 0:
        attr_dict['is_animal_there'] = 1
        dogs = res_df.query("name=='dog'")
        if len(dogs) > 0:
            attr_dict['is_it_a_dog'] = 1
            best_dog = dogs.sort_values(by='confidence', ascending=False).iloc[0]
            coords = (int(best_dog['ycenter']-best_dog['height']//2),
                      int(best_dog['ycenter']+best_dog['height']//2), 
                      int(best_dog['xcenter']-best_dog['width']//2),
                      int(best_dog['xcenter']+best_dog['width']//2))
            dog_crop = cv_image[coords[0]:coords[1], coords[2]:coords[3]]
            classes = classifier_model(tr_pipe(dog_crop).unsqueeze(0).to(device))
            color_cl = classes['color'].argmax().cpu().detach().item() + 1
            tail_cl = classes['tail'].argmax().cpu().detach().item() + 1
            attr_dict['color'] = color_cl
            attr_dict['tail'] = tail_cl
            humans = res_df.query("name=='person'")
            if len(humans) > 0:
                best_dog_coords = best_dog['xcenter'] ,best_dog['ycenter']
                for i, r in persons.iterrows():
                    person_coords = r['xcenter'], r['ycenter']
                    dist = math.dist(person_coords, best_dog_coords)
                    if dist <= 300:
                        attr_dict['is_the_owner_there'] = 1
    all_dicts.append(attr_dict)

In [89]:
pd.DataFrame(all_dicts).to_csv('test_results_4.csv')